<a href="https://colab.research.google.com/github/Arthur-ca/MIE1517/blob/Jing-Guo/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import os
from sklearn.model_selection import train_test_split
import random
import torchvision
from torchvision import datasets, models
from collections import defaultdict
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms.functional as F1
import matplotlib.pyplot as plt
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from PIL import Image
from torch.utils.data import Dataset, DataLoader, TensorDataset, random_split
from torchvision.datasets import ImageFolder
import cv2 as cv
# import tensorflow as tf
import gc
from collections import Counter
import imgaug.augmenters as iaa
import time

In [ ]:
#First link it to google drive and check the subfolder of the data
from google.colab import drive
drive.mount('/content/gdrive')
drive_content = os.listdir('/content/gdrive/My Drive/ColabNotebooks/archive')
print(drive_content)

In [3]:
classeses = []
for i in os.listdir('C:/Users/Admin/Desktop/MIE1517_Project/data'):
    if i!=5:
        classeses.append(i)
classeses

['anger',
 'contempt',
 'disgust',
 'fear',
 'happiness',
 'neutrality',
 'sadness',
 'surprise']

```
Our dataset contains over 5000 photos. Training on this dataset requires significant computing power. However, the best hardware we have is a computer equipped with an Nvidia RTX2080 GPU. To ensure that the model can be trained on the entire dataset, we plan to use 5% of the original data as our training dataset. Below is our code for extracting a small training dataset, a small training function, and the CNN model structure.
```

In [5]:
features_or = []
labels_or = []
train_data_or = []
data_transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
def create_train2(data_percentage=5):
    for i, photo in enumerate(classeses):
        path = os.path.join('C:/Users/Admin/Desktop/MIE1517_Project/data', photo)
        label = i  # Use the index of the class as the label

        # List all images in the class folder
        img_list = os.listdir(path)

        # Calculate the number of images to load (5% of total)
        num_images_to_load = int(len(img_list) * (data_percentage / 100))

        # Randomly select a subset of images
        selected_images = random.sample(img_list, num_images_to_load)


        for img_name in selected_images:
            img_path = os.path.join(path, img_name)
            img_pil = Image.open(img_path)
            transformed_image1 = data_transform1(img_pil)

            if img_pil is None:
                print("Error loading image:", img_path)
                continue

            # Append the original image and its label to the list
            features_or.append(transformed_image1)
            labels_or.append(label)

            train_data_or.append([img_pil, label])

create_train2(data_percentage=5)  # Load only 5% of the data
print(type(features_or[0]))
# Count the occurrences of each label
label_counts_or = Counter(labels_or)

# Print the label counts
for label, count in label_counts_or.items():
    print(f"Label {label}: {count} images")

<class 'torch.Tensor'>
Label 0: 44 images
Label 1: 10 images
Label 2: 21 images
Label 3: 28 images
Label 4: 70 images
Label 5: 26 images
Label 6: 37 images
Label 7: 38 images


In [6]:
# Define a transformation to convert PIL images to PyTorch tensors
data_transform_to_tensor = transforms.Compose([
    transforms.ToTensor()
])

# Convert images and labels to PyTorch tensors
train_images = torch.stack([data_transform_to_tensor(data[0]) for data in train_data_or])
train_labels = torch.tensor([data[1] for data in train_data_or])

# Assuming you have a DataLoader for your training loop
train_dataset = torch.utils.data.TensorDataset(train_images, train_labels)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [7]:
def get_accuracy(model, dataset):
    correct = 0
    total = 0
    for imgs, labels in torch.utils.data.DataLoader(dataset, batch_size=64):
        #############################################
        #To Enable GPU Usage
        if use_cuda and torch.cuda.is_available():
          imgs = imgs.cuda()
          labels = labels.cuda()
        #############################################
        output = model(imgs)
        #select index with maximum prediction score
        pred = output.max(1, keepdim=True)[1]
        correct += pred.eq(labels.view_as(pred)).sum().item()
        total += imgs.shape[0]
    return correct / total

In [8]:
def trainSmall(model, data, batch_size=64, learning_rate=0.01, num_epochs=500):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_acc = []

    # training
    print("Training Start")
    n = 0  # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
            #############################################
            # To Enable GPU Usage
            imgs = imgs.cuda()
            labels = labels.cuda()
            #############################################

            # Check if imgs is already a tensor
            if not torch.is_tensor(imgs):
                img_to_tensor = transforms.ToTensor()
                imgs = img_to_tensor(imgs)

                # Convert input to float
                imgs = imgs.float()

            # Convert the input tensor to the desired type
            imgs = imgs.type(torch.float32)

            out = model(imgs)  # forward pass
            loss = criterion(out, labels)  # compute the total loss
            loss.backward()  # backward pass (compute parameter updates)
            optimizer.step()  # make the updates for each parameter
            optimizer.zero_grad()  # a clean-up step for PyTorch
            n += 1

        train_accuracy = get_accuracy(model, data)
        train_acc.append(train_accuracy)
        print("Epoch:{}, Accuracy:{}".format(epoch, train_accuracy))
    return train_acc

In [9]:
import torch.nn as nn
import torch.nn.functional as F

class CNNLargeNet2(nn.Module):
    def __init__( self, output1=32, output2=64, output3=128, output4=256):
        super(CNNLargeNet2, self).__init__()
        self.name = "CNN"

        # Define the sequential layers for feature extraction
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 5
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )

        # Flatten the tensor for the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024),
            nn.ReLU(inplace=True),
            #nn.Dropout(p=0.5),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            #nn.Dropout(p=0.5),
            nn.Linear(512, 8)
        )

    def forward(self, x):
        x = self.features(x)  # Pass the input through the feature layers
        x = x.view(x.size(0), -1)  # Flatten the tensor for the classifier
        x = self.classifier(x)  # Pass the flattened tensor through the classifier
        return x

In [11]:
use_cuda = True
if torch.cuda.is_available():
    model = CNNLargeNet2().cuda()
Y = trainSmall(model, train_dataset)

Training Start
Epoch:0, Accuracy:0.1386861313868613
Epoch:1, Accuracy:0.10218978102189781
Epoch:2, Accuracy:0.1386861313868613
Epoch:3, Accuracy:0.2591240875912409
Epoch:4, Accuracy:0.15693430656934307
Epoch:5, Accuracy:0.18248175182481752
Epoch:6, Accuracy:0.18613138686131386
Epoch:7, Accuracy:0.20437956204379562
Epoch:8, Accuracy:0.20437956204379562
Epoch:9, Accuracy:0.22262773722627738
Epoch:10, Accuracy:0.22262773722627738
Epoch:11, Accuracy:0.21897810218978103
Epoch:12, Accuracy:0.22627737226277372
Epoch:13, Accuracy:0.22262773722627738
Epoch:14, Accuracy:0.22627737226277372
Epoch:15, Accuracy:0.2518248175182482
Epoch:16, Accuracy:0.21897810218978103
Epoch:17, Accuracy:0.21532846715328466
Epoch:18, Accuracy:0.22992700729927007
Epoch:19, Accuracy:0.21532846715328466
Epoch:20, Accuracy:0.23722627737226276
Epoch:21, Accuracy:0.22992700729927007
Epoch:22, Accuracy:0.24452554744525548
Epoch:23, Accuracy:0.24452554744525548
Epoch:24, Accuracy:0.25547445255474455
Epoch:25, Accuracy:0.259

In [10]:
use_cuda = True
if torch.cuda.is_available():
    model = CNNLargeNet2().cuda()
Y = trainSmall(model, train_dataset, batch_size=64, learning_rate=0.005, num_epochs=260)

Training Start
Epoch:0, Accuracy:0.1386861313868613
Epoch:1, Accuracy:0.2518248175182482
Epoch:2, Accuracy:0.16423357664233576
Epoch:3, Accuracy:0.16058394160583941
Epoch:4, Accuracy:0.1897810218978102
Epoch:5, Accuracy:0.19708029197080293
Epoch:6, Accuracy:0.1897810218978102
Epoch:7, Accuracy:0.20072992700729927
Epoch:8, Accuracy:0.20802919708029197
Epoch:9, Accuracy:0.22262773722627738
Epoch:10, Accuracy:0.21897810218978103
Epoch:11, Accuracy:0.23357664233576642
Epoch:12, Accuracy:0.24087591240875914
Epoch:13, Accuracy:0.2664233576642336
Epoch:14, Accuracy:0.24817518248175183
Epoch:15, Accuracy:0.2591240875912409
Epoch:16, Accuracy:0.2518248175182482
Epoch:17, Accuracy:0.22262773722627738
Epoch:18, Accuracy:0.24452554744525548
Epoch:19, Accuracy:0.22627737226277372
Epoch:20, Accuracy:0.24817518248175183
Epoch:21, Accuracy:0.24087591240875914
Epoch:22, Accuracy:0.2737226277372263
Epoch:23, Accuracy:0.2518248175182482
Epoch:24, Accuracy:0.2956204379562044
Epoch:25, Accuracy:0.270072992

```
************************************************************************
```

In [ ]:
features_or = []
labels_or = []
train_data_or = []
data_transform1 = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])
def create_train2():
    for i, photo in enumerate(classeses):
        path = os.path.join('C:/Users/Admin/Desktop/MIE1517_Project/data', photo)
        label = i  # Use the index of the class as the label

        # Loop over to get every image in the current class
        for img_name in os.listdir(path):
            img_path = os.path.join(path, img_name)
            img_pil = cv.imread(img_path)
            img_pil = Image.fromarray(cv.cvtColor(img_pil, cv.COLOR_BGR2RGB))
            transformed_image1 = data_transform1(img_pil)

            if img_pil is None:
                print("Error loading image:", img_pil)
                continue
            # Append the original image and its label to the list
            features_or.append(transformed_image1)
            labels_or.append(label)

            train_data_or.append([img_pil, label])
create_train2()
print(type(features_or[0]))
# Count the occurrences of each label
label_counts_or = Counter(labels_or)

# Print the label counts
for label, count in label_counts_or.items():
    print(f"Label {label}: {count} images")


In [ ]:

from torchvision import transforms
from collections import defaultdict
labels = labels_or
features = features_or
# Shuffle the labels and features in the same order
combined = list(zip(labels, features))
labels, features = zip(*combined)

# Create a dictionary to store images by label
label_to_images = defaultdict(list)

# Group images by label
for label, img in zip(labels, features):
    label_to_images[label].append(img)

# Set the ratios for train, validation, and test sets
train_ratio = 0.6
validation_ratio = 0.2
test_ratio = 0.2

train_data = []
validation_data = []
test_data = []

# Split data for each label
for label, images in label_to_images.items():
    random.shuffle(images)  # Shuffle the images for each label

    num_images = len(images)
    num_train = int(train_ratio * num_images)
    num_validation = int(validation_ratio * num_images)
    num_test = num_images - num_train - num_validation

    train_data.extend([(img, label) for img in images[:num_train]])
    validation_data.extend([(img, label) for img in images[num_train:num_train + num_validation]])
    test_data.extend([(img, label) for img in images[num_train + num_validation:]])

In [ ]:

class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img, label = self.data[index]

        # Convert img to PyTorch tensor
        img_tensor = torch.tensor(img, dtype=torch.float32)

        return img_tensor, label

# Create datasets
train_dataset = CustomDataset(train_data)
validation_dataset = CustomDataset(validation_data)
test_dataset = CustomDataset(test_data)

# Create DataLoader instances
batch_size = 64  # You can adjust this based on your needs
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the tensor size
for images, labels in train_loader:
    print(images.size())  # This will print the size of the tensor
    break


```
Old Model and train func
```

In [ ]:
class CNNLargeNet(nn.Module):
    def __init__(self, output1=32, output2=64, output3=128, output4=256):
        super(CNNLargeNet, self).__init__()
        self.name = "large"
        
        # Define the sequential layers
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(3, output1, kernel_size=7, stride=1, padding=3),
            nn.BatchNorm2d(output1),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 2
            nn.Conv2d(output1, output2, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(output2),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 3
            nn.Conv2d(output2, output3, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(output3),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 4
            nn.Conv2d(output3, output4, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(output4),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.3),
            nn.MaxPool2d(2, 2)
        )
        
        # Flatten the tensor for the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(output4 * 14 * 14, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            
            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            
            nn.Linear(64, 8)  # Assuming 8 output classes
        )

    def forward(self, x):
        x = self.features(x)  # Pass the input through the feature layers
        x = x.view(x.size(0), -1)  # Flatten the tensor for the classifier
        x = self.classifier(x)  # Pass the flattened tensor through the classifier
        return x

In [ ]:
def trainmodel(model, train_dataset, val_dataset, batch=32, learningRate=0.01, num_epochs=500):
    # Create loss function and optimizer.
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    # Load data
    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=True)
    # Create accuracy lists
    iters, losses, train_accuracy, validation_accuracy = [], [], [], []
    n = 0  # Initialize the iteration counter
    # Train the data
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_data in train_loader:
            imgs, labels = batch_data
            # img_to_tensor = transforms.ToTensor()
            # imgs = img_to_tensor(image)
            if use_cuda and torch.cuda.is_available():
                imgs = imgs.cuda()
                labels = labels.cuda()

            out = model(imgs)  # Forward pass
            loss = criterion(out, labels)  # Compute the total loss
            loss.backward()  # Backward pass (compute parameter updates)
            optimizer.step()  # Make the updates for each parameter
            optimizer.zero_grad()  # A clean-up step for PyTorch
            n += 1
            # print("Input image data type:", imgs.dtype)
            # print("Model parameters data types:")
            # for param in model.parameters():
            #     print(param.dtype)
            # Save the current training information
            running_loss += loss.item()

        iters.append(n)
        losses.append(running_loss / len(train_loader))  # Compute average loss for the epoch
        train_accuracy.append(get_accuracy(model, train_dataset))  # Compute training accuracy

        # Compute validation accuracy for this epoch
        validation_accuracy.append(get_accuracy(model, val_dataset))  # Compute validation accuracy

        # Print progress for the epoch
        print("Epoch {}, Train Accuracy: {:.6f}%, Validation Accuracy: {:.6f}%".format(
            epoch, train_accuracy[epoch], validation_accuracy[epoch]))

        # Save model checkpoint
        model_checkpoint_path = 'C:/Users/Admin/Desktop/MIE1517_Project/output'  # You can change the directory as needed
        model_checkpoint_file = os.path.join(model_checkpoint_path, f'{model.name} =Batch_size_{batch}_model_epoch{epoch + 1}.pth')
        torch.save(model.state_dict(), model_checkpoint_file)
        print(f"Saved model checkpoint: {model_checkpoint_file}")

    print("Finished Training")
    return iters, losses, train_accuracy, validation_accuracy

```
End Old Model and train func
```

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class CNNLargeNet2(nn.Module):
    def __init__( self, output1=32, output2=64, output3=128, output4=256):
        super(CNNLargeNet2, self).__init__()
        self.name = "CNN"
        
        # Define the sequential layers for feature extraction
        self.features = nn.Sequential(
            # Convolutional Layer 1
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),
            
            # Convolutional Layer 2
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 3
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 4
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2),

            # Convolutional Layer 5
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.BatchNorm2d(512),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, 2)
        )
        
        # Flatten the tensor for the fully connected layers
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Linear(512, 8)
        )

    def forward(self, x):
        x = self.features(x)  # Pass the input through the feature layers
        x = x.view(x.size(0), -1)  # Flatten the tensor for the classifier
        x = self.classifier(x)  # Pass the flattened tensor through the classifier
        return x

In [ ]:
def get_model_name(name, batch_size, learning_rate, epoch):
    path = "model_{0}_bs{1}_lr{2}_epoch{3}".format(name, batch_size, learning_rate, epoch)
    return path

def evaluate(model, loader):
    """ Evaluate the network """
    correct = 0
    total = 0

    for inputs, labels in loader:
      if use_cuda and torch.cuda.is_available():
        inputs = inputs.cuda()
        labels = labels.cuda()

      outputs = model(inputs)
      predicted = outputs.max(1, keepdim=True)[1]
      total += inputs.shape[0]
      correct += predicted.eq(labels.view_as(predicted)).sum().item()
    return correct / total


In [ ]:
def trainmodel2(model, train_dataset, val_dataset, batch=64, learningRate=0.01, num_epochs=300):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learningRate)
    
    model_checkpoint_path = 'C:/Users/Admin/Desktop/MIE1517_Project/output'  # You can change the directory as needed

    train_loader = DataLoader(train_dataset, batch_size=batch, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch, shuffle=False) 

    train_accuracy = np.zeros(num_epochs)
    train_losses = np.zeros(num_epochs)
    validation_accuracy = np.zeros(num_epochs)
    validation_losses = np.zeros(num_epochs)
    iters = []

    # Check CUDA availability
    use_cuda = torch.cuda.is_available()
    if use_cuda:
        model = model.cuda()

    n = 0  # Initialize the iteration counter
    # Training
    print("Training Begin...\n")
    start_time = time.time()
    for epoch in range(num_epochs):
        # Train mode
        model.train()  # Set the model to training mode
        running_loss = 0.0
        
        for i, data in enumerate(train_loader, 0):
            # Get the inputs
            inputs, labels = data
            # Set up for gpu running
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()
            # Zero the parameter gradients
            optimizer.zero_grad()
            # Forward pass, backward pass, and optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            n += 1
            # Calculate loss
            running_loss += loss.item()

        train_losses[epoch] = running_loss
        train_accuracy[epoch] = evaluate(model, train_loader)
        
        # Evaluation mode
        model.eval()  # Set the model to evaluation mode for accuracy computation
        valid_loss = 0.0
        # Running without gradients are computed and model weights update
        for inputs, labels in val_loader:
            if use_cuda and torch.cuda.is_available():
                inputs = inputs.cuda()
                labels = labels.cuda()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            valid_loss += loss.item()

        validation_losses[epoch] = valid_loss
        validation_accuracy[epoch] = evaluate(model, val_loader)
        iters.append(n)
        
        # Print progress for the epoch
        print(("Epoch {}: Train Accuracy: {}, Train loss: {} |"+
               "Validation Accuracy: {}, Validation loss: {}").format(
                   epoch + 1,
                   train_accuracy[epoch],
                   train_losses[epoch],
                   validation_accuracy[epoch],
                   validation_losses[epoch]))

        # Save model checkpoint every 5 epochs
        # if (epoch + 1) % 5 == 0:  # +1 because epochs start from 0
            # Save the current model checkpoint to a file
        model_path = get_model_name(model.name, batch_size, learningRate, epoch)
        model_path = os.path.join(model_checkpoint_path, model_path)
        torch.save(model.state_dict(), model_path)

    print("Finshied Training")
    end_time = time.time()
    elapsed_time = end_time - start_time
    print("Total time elapsed: {:.2f} seconds".format(elapsed_time))
    # Write the train/test loss/err into CSV file for plotting later
    # Save model checkpoint
    epochs = np.arange(1, num_epochs + 1)
    np.savetxt("{}_train_acc.csv".format(model_path), train_accuracy)
    np.savetxt("{}_train_loss.csv".format(model_path), train_losses)
    np.savetxt("{}_val_acc.csv".format(model_path), validation_accuracy)
    np.savetxt("{}_val_loss.csv".format(model_path), validation_losses)

    plt.title("Training Loss Curve")
    plt.plot(iters, train_losses, label="Train")
    plt.xlabel("iters")
    plt.ylabel("Loss")
    plt.show()

    plt.title("Accuracy Curve")
    plt.plot(iters, train_accuracy, label="Train")
    plt.xlabel("iters")
    plt.ylabel("Accuracy")
    plt.show()
    print("Finished Training")
    return iters, train_losses, train_accuracy, validation_accuracy

In [ ]:
def trainSmall(model, data, batch_size=32, learning_rate=0.01, num_epochs=260):
    train_loader = torch.utils.data.DataLoader(data, batch_size=batch_size)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    train_acc = []

    # training
    print("Training Start")
    n = 0  # the number of iterations
    for epoch in range(num_epochs):
        for imgs, labels in iter(train_loader):
            #############################################
            # To Enable GPU Usage
            imgs = imgs.cuda()
            labels = labels.cuda()
            #############################################

            # Check if imgs is already a tensor
            if not torch.is_tensor(imgs):
                img_to_tensor = transforms.ToTensor()
                imgs = img_to_tensor(imgs)

                # Convert input to float
                imgs = imgs.float()

            # Convert the input tensor to the desired type
            imgs = imgs.type(torch.float32)

            out = model(imgs)  # forward pass
            loss = criterion(out, labels)  # compute the total loss
            loss.backward()  # backward pass (compute parameter updates)
            optimizer.step()  # make the updates for each parameter
            optimizer.zero_grad()  # a clean-up step for PyTorch
            n += 1

        train_accuracy = get_accuracy(model, data)
        train_acc.append(train_accuracy)
        print("Epoch:{}, Accuracy:{}".format(epoch, train_accuracy))
    return train_acc


In [ ]:
batch_size = 3
num_workers = 1
small_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, num_workers=num_workers, shuffle=True)

# Visualize some sample data
classes = ['neutrality', 'sadness', 'fear', 'surprise', 'contempt', 'happiness', 'anger', 'disgust']
dataiter = iter(small_loader)
images, labels = next(dataiter)
images = images.numpy() # convert images to numpy for display

# plot the images in the batch, along with the corresponding labels
fig = plt.figure(figsize=(25, 4))
for idx in np.arange(3):
    ax = fig.add_subplot(2, 10, idx+1, xticks=[], yticks=[])

    # If the image has three channels
    if images[idx].shape[0] == 3:
        plt.imshow(np.transpose(images[idx], (1, 2, 0)))
    else:  # If the image is grayscale
        plt.imshow(images[idx][0], cmap='gray')

    ax.set_title(classes[labels[idx]])

In [ ]:
use_cuda = True
if torch.cuda.is_available():
    model = CNNLargeNet2().cuda()
Y = trainmodel2(model, train_data, validation_data, batch=64, learningRate=0.01, num_epochs=300)